In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
from scipy.signal import welch
from pyFAST.input_output import FASTOutputFile

In [2]:
casedates = [
    'April5',
    'May15',
    'June3',
]
conditions = {
    #'MS': 'mon',
    'MS+veer': 'mon_veer',
    'LLJ': 'llj',
}
prefix = 'iea15mw'
suffix = '_classC'
Nhour = 18 # expected simulations (hourly) per case
Nseed = 6 # turbsim seeds

# simulation inputs
TMax = 720.0 # OpenFAST driver
sampleheights = [80,30,150,270] # InflowWind

In [3]:
QOIs = [
    'GenPwr_[kW]',
    'GenTq_[kN-m]',
    'GenSpeed_[rpm]',
    'RtSpeed_[rpm]', # Rotor (low-speed shaft) azimuth angular speed
    'BldPitch1_[deg]',
    'RtTSR_[-]',
    'RtFldFxh_[N]', # Total rotor aerodynamic/hydrodynamic and buoyant load (force in x direction)
    'RtFldFyh_[N]', # Total rotor aerodynamic/hydrodynamic and buoyant load (force in y direction)
    'RtFldFzh_[N]', # Total rotor aerodynamic/hydrodynamic and buoyant load (force in z direction)
    'RootMxb1_[kN-m]', # Blade 1 edgewise moment (i.e., the moment caused by edgewise forces) at the blade root
    'RootMyb1_[kN-m]', # Blade 1 flapwise moment (i.e., the moment caused by flapwise forces) at the blade root
    'YawBrMxp_[kN-m]', # _Nonrotating_ tower-top / yaw bearing roll moment (NOTE: Tanmoy's analysis was in a rotating nacelle frame of reference)
    'YawBrMyp_[kN-m]', # _Nonrotating_ tower-top / yaw bearing pitch moment (NOTE: Taoy's analysis was in a rotating nacelle frame of reference)
    'YawBrMzp_[kN-m]', # Tower-top / yaw bearing yaw moment
    'LSSTipMys_[kN-m]', # _Nonrotating_ low-speed shaft bending moment at the shaft tip (teeter pin for 2-blader, apex of rotation for 3-blader) (NOTE: Tanmoy's analysis was in a rotating frame of reference)
    'LSSTipMzs_[kN-m]', # _Nonrotating_ low-speed shaft bending moment at the shaft tip (teeter pin for 2-blader, apex of rotation for 3-blader) (NOTE: Tanmoy's analysis was in a rotating frame of reference)
    'TwrBsMxt_[kN-m]', # Tower base roll (or side-to-side) moment (i.e., the moment caused by side-to-side forces)
    'TwrBsMyt_[kN-m]', # Tower base pitching (or fore-aft) moment (i.e., the moment caused by fore-aft forces)
    'TwrBsMzt_[kN-m]', # Tower base yaw (or torsional) moment
]

In [4]:
def process_sim(outfile,period=600):
    """Process FAST output over the last `period` seconds"""
    output = FASTOutputFile(outfile)
    df = output.toDataFrame()
    df = df.set_index('Time_[s]')
    if df.index[-1] != TMax:
        print('WARNING: SOLUTION DID NOT CONVERGE?!?')
        return None
    
    IfWsamples = [col for col in df.columns if col.startswith('Wind')]
    df = df[QOIs + IfWsamples]
    
    # trim output to last 10 min
    df = df.loc[slice(TMax-period,None)]

    # power should be positive
    assert df['GenPwr_[kW]'].mean() >= 0
    
    # calculate statistics
    dfmean = df.mean().to_frame().T
    dfmean['t'] = hr*3600
    dfmean = dfmean.set_index('t')
    dfmean = dfmean.rename(columns={channel:'Mean'+channel
                                    for channel in dfmean.columns})
    dfstd = df.std().to_frame().T
    dfstd['t'] = hr*3600
    dfstd = dfstd.set_index('t')
    dfstd = dfstd.rename(columns={channel:'Stdev'+channel
                                  for channel in dfstd.columns})
    dfmin = df.min().to_frame().T
    dfmin['t'] = hr*3600
    dfmin = dfmin.set_index('t')
    dfmin = dfmin.rename(columns={channel:'Min'+channel
                                  for channel in dfmin.columns})
    dfmax = df.max().to_frame().T
    dfmax['t'] = hr*3600
    dfmax = dfmax.set_index('t')
    dfmax = dfmax.rename(columns={channel:'Max'+channel
                                  for channel in dfmax.columns})
    df = pd.concat([dfmean,dfstd,dfmin,dfmax],axis=1)
    
    return df

In [6]:
%%time
results = {}

for cond,dname in conditions.items():
    results[cond] = {}
    for date in casedates:
        dpath = f'{date}{suffix}/{dname}{suffix}'
        results[cond][date] = {}
        for seed in range(Nseed):
            results[cond][date][seed] = {}
            dflist = []
            for hr in range(Nhour):
                outfile = f'{dpath}/{prefix}_{hr:02d}_seed{seed}.outb'
                print(outfile,f': hour {hr}, realization {seed}')
                
                # read FAST binary data, convert to mean, stdev
                df = process_sim(outfile)
                if df is not None:
                    dflist.append(df)
            results[cond][date][seed] = pd.concat(dflist, axis=0)

# CPU times: user 30min 49s, sys: 2min 12s, total: 33min 2s
# Wall time: 33min 3s

April5_classC/mon_veer_classC/iea15mw_00_seed0.outb : hour 0, realization 0
April5_classC/mon_veer_classC/iea15mw_01_seed0.outb : hour 1, realization 0
April5_classC/mon_veer_classC/iea15mw_02_seed0.outb : hour 2, realization 0
April5_classC/mon_veer_classC/iea15mw_03_seed0.outb : hour 3, realization 0
April5_classC/mon_veer_classC/iea15mw_04_seed0.outb : hour 4, realization 0
April5_classC/mon_veer_classC/iea15mw_05_seed0.outb : hour 5, realization 0
April5_classC/mon_veer_classC/iea15mw_06_seed0.outb : hour 6, realization 0
April5_classC/mon_veer_classC/iea15mw_07_seed0.outb : hour 7, realization 0
April5_classC/mon_veer_classC/iea15mw_08_seed0.outb : hour 8, realization 0
April5_classC/mon_veer_classC/iea15mw_09_seed0.outb : hour 9, realization 0
April5_classC/mon_veer_classC/iea15mw_10_seed0.outb : hour 10, realization 0
April5_classC/mon_veer_classC/iea15mw_11_seed0.outb : hour 11, realization 0
April5_classC/mon_veer_classC/iea15mw_12_seed0.outb : hour 12, realization 0
April5_cl

In [51]:
# convert nested dict to netcdf
#   results[cond][date][seed]
all_sims = []
for cond,dname in conditions.items():
    for date in casedates:
        df = pd.concat(results[cond][date],names=['turb_seed','t_sec'])
        ds = df.to_xarray().expand_dims(date=[date],cond=[cond])
        all_sims.append(ds)
all_sims = xr.merge(all_sims)

In [62]:
newvarnames = {}
newunits = []
varnames = list(all_sims.data_vars)
for varn in varnames:
    varnsplit = varn.split('_')
    assert len(varnsplit) == 2
    newvarnames[varn] = varnsplit[0]
    units = varnsplit[1]
    assert (units[0] == '[') and (units[-1] == ']')
    newunits.append(units[1:-1])

In [65]:
all_sims = all_sims.rename_vars(newvarnames)
for varn,units in zip(all_sims.data_vars, newunits):
    all_sims[varn].attrs['units'] = units

In [67]:
all_sims.to_netcdf('openfast_monveer_llj_classC_sim_stats.nc')